Extracts car wise features. Does not write to h5

In [1]:
using AutomotiveDrivingModels
using AutoRisk
using HDF5
using NGSIM

INFO: Recompiling stale cache file /home/raunak/.julia/lib/v0.6/AutoRisk.ji for module AutoRisk.


## Caution: The below `build_feature_extractor` needs `AutoRisk` to be on laneid branch if you are including the laneidfeatureextractor

In [2]:
function build_feature_extractor()
    subexts = [
        LaneIDFeatureExtractor(),
        CoreFeatureExtractor(),
        TemporalFeatureExtractor(),
        WellBehavedFeatureExtractor(),
        CarLidarFeatureExtractor(20, carlidar_max_range = 50.),
        ForeForeFeatureExtractor()
    ]
    ext = MultiFeatureExtractor(subexts)
    return ext
end

build_feature_extractor (generic function with 1 method)

In [9]:
function extract_features(
        ext,
        trajdata, 
        roadway, 
        timestep_delta, 
        record_length, 
        offset, 
        prime,
        maxframes)
    n_features = length(ext)
    max_n_objects = maximum(n_objects_in_frame(trajdata, i) for i in 1 : nframes(trajdata))
    scene = Scene(max_n_objects)
    rec = SceneRecord(record_length, 0.1, max_n_objects)
    features = Dict{Int, Array{Float64}}()
    ctr = 0
    n_frames = nframes(trajdata)
    
    for frame in (offset - prime : offset - 1)
        # prime the rec
        AutomotiveDrivingModels.update!(rec, get!(scene, trajdata, frame))
    end

    veh_features = pull_features!(ext, rec, roadway, 1)
#     println("veh_features shape = $(size(veh_features))")

    for frame in offset : (n_frames - offset)
#     for frame in offset : offset+1
        ctr += 1
#         println("ctr = $(ctr)")
        if maxframes != nothing && ctr >= maxframes
            break
        end

        print("\rframe $(frame) / $(n_frames - offset)")
            
        # update the rec
        AutomotiveDrivingModels.update!(rec, get!(scene, trajdata, frame))

        # every timestep_delta step, extract features
        if frame % timestep_delta == 0
            for (vidx, veh) in enumerate(scene)
                # extract features
                veh_features = pull_features!(ext, rec, roadway, vidx)
                #println("veh.id = $(veh.id) \n")
                
                # add entry to features if vehicle not yet encountered
                if !in(veh.id, keys(features))
                    features[veh.id] = zeros(n_features, 0)
                end

                # stack onto existing features
                features[veh.id] = cat(2, features[veh.id], 
                    reshape(veh_features, (n_features, 1)))
            end
        end
    end
    
    
#     println("shape features of vehicle 23= $(size(features[23]))")
#     println("length of keys of features i.e total number 
#         of distinct vehicles encountered \n $(length(keys(features)))")
    
    # Print the shape of the associated feature for every vehicle index
#     for (key,value) in features
#         println("key: $(key) ==> shape: $(size(value))")
#     end
        
    return features
end

extract_features (generic function with 1 method)

In [10]:
        timestep_delta = 1 # timesteps between feature extractions
        record_length = 10 # number of frames for record to track in the past
        offset = 500 # from ends of the trajectories
        prime = 10
        maxframes = nothing # nothing for no max

    ext = build_feature_extractor()
    features = Dict{Int, Dict{Int, Array{Float64}}}()

    tic()
    # extract 


        # setup
        trajdata = load_trajdata(1)
        roadway = get_corresponding_roadway(1)
        features = extract_features(
            ext, 
            trajdata, 
            roadway, 
            timestep_delta, 
            record_length, 
            offset, 
            prime,
            maxframes
        )
    toc()


frame 9036 / 9036elapsed time: 807.055164695 seconds


807.055164695

In [11]:
# This features thing is gold. It is the ngsim data extracted carwise above
length(keys(features))

2150

In [36]:
function calcLaneChanges(test::Array{Float64})
lanechanges = 0
for i = 1:length(test)-1
    if test[i] != test[i+1]
        lanechanges = lanechanges + 1
    end
end
return lanechanges
end

calcLaneChanges (generic function with 1 method)

In [38]:
calcLaneChanges(features[233][1,:])

4

In [39]:
laneChangeData = Array{Float64}
for (key,value) in features
#    if verbose
#         println("size of features = $(size(value))")
#     end
    laneData = value[1,:] # Note that 1 is the index of lane id 
    numLaneChanges = calcLaneChanges(laneData)
    timeHorizon = length(laneData)
    
    println("$(numLaneChanges) $(timeHorizon)")
end

3 271
2 442
2 840
2 401
2 517
3 656
4 323
2 515
5 812
2 431
2 592
3 697
2 741
4 556
3 582
3 473
2 463
2 422
2 673
2 649
2 471
2 428
3 470
2 598
2 679
2 466
2 530
2 543
2 676
2 451
2 581
9 359
2 564
2 451
3 474
2 856
2 711
2 960
2 453
2 574
2 345
4 706
2 495
2 510
3 430
2 509
2 683
2 643
2 458
2 507
5 509
4 968
2 776
2 690
2 526
2 919
2 435
2 507
2 760
1 204
2 468
2 504
3 553
2 674
2 290
2 474
2 469
2 554
2 447
2 526
2 172
2 678
3 441
2 427
2 677
2 742
2 965
2 499
2 453
11 499
1 431
2 468
2 562
2 804
1 125
3 463
2 516
0 137
2 601
2 399
2 453
5 565
2 553
2 511
2 430
2 470
2 451
2 424
2 457
2 660
1 310
4 460
2 742
2 639
2 370
2 782
2 305
3 850
2 519
2 608
2 724
2 470
2 489
3 546
2 598
2 924
3 549
2 565
2 424
5 471
2 524
2 948
2 520
2 584
2 472
2 507
10 655
3 351
3 380
2 426
2 653
2 508
2 671
3 375
2 937
3 544
2 936
2 445
2 551
2 509
2 590
1 211
2 508
0 4
2 442
2 487
0 47
2 471
2 477
4 666
2 550
2 720
2 575
5 503
2 491
2 488
2 713
2 567
4 514
3 820
2 466
2 286
2 500
2 378
2 470
2 417
2 856

2 550
2 510
2 555
2 581
3 846
2 770
2 459
2 512
2 426
2 447
2 380
2 541
2 944
2 514
2 492
2 443
2 591
2 610
2 476
2 685
2 754
2 530
3 320
3 495
1 574
2 690
2 502
2 339
2 503
3 423
2 927
2 605
2 422
3 961
2 471
2 517
3 835
2 292
2 514
2 562
1 256
2 385
3 462
2 663
2 255
3 485
2 422
2 554
2 434
2 433
2 664
2 448
2 295
2 454
2 464
2 455
2 506
2 500
1 535
2 629
5 385
2 516
2 591
2 489
2 556
3 482
1 672
3 547
2 476
1 456
4 520
2 486
2 445
2 453
2 825
0 78
2 559
2 457
2 496
2 680
2 683
1 245
2 712
4 388
2 821
2 447
2 821
2 565
2 565
2 945
3 409
2 450
2 552
0 71
2 439
2 611
2 805
2 744
1 497
2 559
2 240
2 802
2 902
2 570
2 513
2 644
2 721
2 429
4 443
1 425
1 649
2 710
2 289
0 306
2 951
2 479
0 274
2 592
2 756
0 99
2 409
2 335
2 421
3 431
2 642
2 501
3 462
2 650
2 650
3 504
2 427
2 528
2 506
2 939
2 482
2 466
2 435
2 552
2 425
2 489
2 564
2 514
2 504
2 580
2 720
2 452
2 532
2 601
2 932
2 512
6 478
3 378
2 501
2 505
2 347
1 350
4 533
2 276
2 542
2 545
3 850
3 517
2 468
2 483
2 469
5 545
2 433
1

In [45]:
println(features[76][1,:])

[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0]


In [46]:
function calcTimeGapAveragedOverHorizon(timegapdata::Array{Float64})
    timegapsum = 0.0
    timegapnum = 0.0
    for i = 1:length(timegapdata)
        
        # Don't count the huge timegap assigned when no car in front
        if timegapdata[i] != 30.0
            timegapsum = timegapsum + timegapdata[i]
            timegapnum = timegapnum + 1
        end
    end
    
    return timegapsum/timegapnum
end

calcTimeGapAveragedOverHorizon (generic function with 1 method)

In [47]:
# print the timegaps
# timegap will be 16 index. Recall laneid was index 1

# TAKE CARE ABOUT THE 30.0 VALUES IN TIMEGAP
# NEED TO CHECK IN BOTH NGSIM AND OUR TRAINED POLICY SIMULATIONS

for (key,value) in features
    timegapdata = value[16,:]
    # Calculate the average timegap over horizon for every agent
    println("$(calcTimeGapAveragedOverHorizon(timegapdata))")
end

1.156528172870694
1.2901632162430652
3.1676253867815616
2.893715641358782
3.1538965119701445
1.9774831893349183
0.9799054563096302
1.3205851258516135
2.428039230377182
0.9199319389760084
1.5044026814842046
1.1222851166853105
3.009385492800137
1.1220696885100776
1.7991776553936625
1.3525255943164656
1.3708875159986094
3.4739654679199137
3.471782171161586
1.0397317909492714
1.760901654451472
1.523926642867148
2.096715895377784
1.2465482686618896
1.1079664270373386
2.0444686673960026
3.1048737816936
2.6248212845059364
1.5920596344555753
2.1621709014397372
1.8328401188548986
0.8926353769488307
3.2259377038653483
2.2893802966578445
2.4178801880448044
3.6158162235898583
1.6353897677417535
2.1662500466834795
1.4832442515382838
1.4475131910154495
1.2316523829487438
1.2763209819466925
2.0355086749645004
2.2882197648077116
1.1118897672029144
1.3878056329267157
1.2441499389201216
1.786771646630295
2.212167579954674
2.193574753681707
2.7903036404047854
2.9530440868474295
0.882167288962494
3.056082

2.242502481095034
1.7364108897090587
4.9646755923829025
1.3859752105550567
2.7018312050005258
3.416789268759939
1.84765733121176
0.7309690211993489
1.2525404724724565
1.7009663568255549
2.3664338385584562
2.2606015638734056
2.0705944669598195
2.506660355436327
1.3755867883800448
1.3572220790428227
3.211688361160557
4.553161020116916
1.6830724791260574
3.703849789203633
1.5680738624476145
1.8982957715785158
1.9634059833252555
1.0638426618201158
1.5128406158875862
2.5472718576644158
1.3899834791938939
2.478195410069339
1.641106052800365
1.957424828252248
3.0263364719216224
1.4261524387286646
1.2332671486483848
0.7113714989461275
1.2141428638758773
2.396930433386451
1.347291873907252
1.9653345968526514
1.3557983516387788
1.4066812199639693
0.8191268552382498
1.6835903378903776
1.1625263078012136
0.6541899524936385
1.7827464534610287
1.5947113395370958
3.0879859634370996
1.3393178216857218
3.752156350002595
1.6767767642296794
2.7978470494871606
1.919336499567498
0.9219402319784079
2.972434

1.984869243079593
1.9607903832434932
2.3772857098061713
2.5065274094868952
1.427627145871049
3.526516543821682
2.273019275293887
2.0684036160118544
1.2885520615159263
1.6302648909189754
2.256022335616852
2.89074776010919
1.2762072851490578
1.8166475229963606
1.470476518918453
2.5555502150430844
3.471326248016489
3.1846561701434375
1.4311898995060364
1.9374579931205218
1.4635988498947539
0.874968397890098
3.271807070301258
2.370351831938244
1.6376766325168475
1.292337281238007
2.6858643664982598
2.503421691151908
2.8102924594387275
3.353364804968938
1.9033032639223453
1.8938839349494594
2.2563184046292624
1.9624681784156217
2.6201963459974618
1.7858856975508672
1.2783545339345916
2.9022165554648542
2.4630275884136874
0.9580594749797027
2.490448352370006
3.1647112879853188
0.844656562877298
0.966263279603042
3.4954104318061776
2.3341711672688015
1.6389911822885104
2.0615548267881176
2.5699195033323705
1.924044333739589
1.2049144805671077
2.7085813218477863
2.6860363046686198
1.3289354473

1.5889027613253264
1.0585647999751013
2.0705725772059234
1.8332874695856045
1.5684376997917788
3.364855496145269
1.512118810918282
1.493105232298806
1.3770649171607345
1.9753393745340388
0.9333451906775085
1.512707283771518
1.3384313844823872
1.5479494331544215
1.4490324088154445
1.3865600090584522
4.16563780031189
2.7356423013644626
2.766915080328214
1.4069538241404291
1.0513581884529493
1.7882326544086675
1.1878397457122702
1.123271928118784
2.3100611783135028
2.671036238618676
2.6055557650832166
2.0415063720906628
1.7598961195751666
1.7607977649530566
2.7119522123379096
1.7187848976253692
1.9325547125358968
2.8876202273835037
3.4429739426624244
1.7473151067094208
2.9858649812546996
1.2725671353641996
2.1087555986780386
1.3806993565641632
2.789159606492963
2.4286891224450833
2.003784647908816
1.3278919716198176
0.7781805824031782
1.7620593106487732
2.209796175188833
2.2338217412672274
1.0417919733808307
2.0338851861433587
2.0303392079083773
0.9040086755717561
4.270992831808038
2.1176